Làm sạch dữ liệu

In [1]:
import pandas as pd
import numpy as np

# Đọc dữ liệu
df = pd.read_csv('GIS.DataAQI.csv')

# Xử lý giá trị thiếu (N/A)
df.replace('N/A', np.nan, inplace=True)

# Chuyển các cột số về kiểu số
numeric_cols = ['aqi', 'pm25', 'pm10', 'no2', 'co', 'o3', 'so2', 
                'humidity', 'temperature', 'wind', 'population', 'area']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Thêm cột mới để phân loại các vùng
df['region'] = np.where(df['country'] == 'Vietnam', 'Vietnam', 'Denmark')

# Xử lý ngày tháng
df['update_time'] = pd.to_datetime(df['update_time'], errors='coerce')
df['saved_at'] = pd.to_datetime(df['saved_at'], errors='coerce')

Tạo các đặc trưng mới

In [2]:
# AQI to Health Risk (dựa trên tiêu chuẩn EPA)
def aqi_to_risk(aqi):
    if pd.isna(aqi) or aqi == '-':
        return np.nan
    aqi = float(aqi)
    if aqi <= 50:
        return 0  # Tốt (Good) - Xanh lá
    elif aqi <= 100:
        return 1  # Trung bình (Moderate) - Vàng
    elif aqi <= 150:
        return 2  # Không lành mạnh cho nhóm nhạy cảm (Unhealthy for Sensitive Groups) - Cam
    elif aqi <= 200:
        return 3  # Không lành mạnh (Unhealthy) - Đỏ
    elif aqi <= 300:
        return 4  # Rất không lành mạnh (Very Unhealthy) - Tím
    else:
        return 5  # Nguy hiểm (Hazardous) - Nâu

# Áp dụng hàm trên cột AQI
df['health_risk'] = df['aqi'].apply(aqi_to_risk)

# Tạo các đặc trưng cho mật độ dân số
df['population_density'] = df['population'] / df['area']

# Tạo biến chỉ ra mức độ ô nhiễm của các thông số riêng biệt
pm25_thresholds = [0, 12, 35.4, 55.4, 150.4, 250.4, 500.4]
pm10_thresholds = [0, 54, 154, 254, 354, 424, 604]
no2_thresholds = [0, 53, 100, 360, 649, 1249, 2049]
so2_thresholds = [0, 35, 75, 185, 304, 604, 1004]
o3_thresholds = [0, 54, 70, 85, 105, 200, 300]

# Tạo hàm phân loại chung
def categorize_pollutant(value, thresholds):
    if pd.isna(value) or value == '-':
        return np.nan
    value = float(value)
    for i, threshold in enumerate(thresholds[1:], 1):
        if value < threshold:
            return i-1
    return len(thresholds) - 1

# Áp dụng cho từng chất ô nhiễm
df['pm25_level'] = df['pm25'].apply(lambda x: categorize_pollutant(x, pm25_thresholds))
df['pm10_level'] = df['pm10'].apply(lambda x: categorize_pollutant(x, pm10_thresholds))
df['no2_level'] = df['no2'].apply(lambda x: categorize_pollutant(x, no2_thresholds))
df['so2_level'] = df['so2'].apply(lambda x: categorize_pollutant(x, so2_thresholds))
df['o3_level'] = df['o3'].apply(lambda x: categorize_pollutant(x, o3_thresholds))

 Phân tích thống kê mô tả

In [3]:
# Thống kê cơ bản theo quốc gia
stats_by_country = df.groupby('country')[numeric_cols].mean()

# Tính tỷ lệ các mức rủi ro sức khỏe theo quốc gia
risk_distribution = df.groupby(['country', 'health_risk']).size().unstack()

# Tìm thành phố có mức ô nhiễm cao nhất và thấp nhất
worst_cities = df.sort_values('aqi', ascending=False).head(10)
best_cities = df[df['aqi'] > 0].sort_values('aqi').head(10)

XÂY DỰNG MÔ HÌNH ĐÁNH GIÁ RỦI RO SỨC KHỎE

Chuẩn bị dữ liệu cho mô hình

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Chọn các đặc trưng và biến mục tiêu
features = ['pm25', 'pm10', 'no2', 'co', 'o3', 'so2', 'humidity', 'temperature', 'wind']
X = df[features]

# Biến mục tiêu: health_risk (đã tính dựa trên AQI)
y = df['health_risk']

# Loại bỏ các hàng không có giá trị mục tiêu
valid_rows = y.notna()
X = X[valid_rows]
y = y[valid_rows]

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tiền xử lý dữ liệu
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, features)
    ])

# Pipeline dùng cho huấn luyện mô hình
preprocessed_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Áp dụng xử lý
X_train_processed = preprocessed_pipeline.fit_transform(X_train)
X_test_processed = preprocessed_pipeline.transform(X_test)

 Huấn luyện mô hình phân loại

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Huấn luyện mô hình Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_processed, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test_processed)
print(classification_report(y_test, y_pred))

# Lưu mô hình
import joblib
joblib.dump(model, 'health_risk_model.pkl')
joblib.dump(preprocessed_pipeline, 'preprocessor.pkl')

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       185
         1.0       1.00      0.98      0.99        65
         2.0       1.00      1.00      1.00        30
         3.0       1.00      1.00      1.00         5

    accuracy                           1.00       285
   macro avg       1.00      1.00      1.00       285
weighted avg       1.00      1.00      1.00       285



['preprocessor.pkl']

In [6]:
from health_recommendations import get_health_recommendations, get_specific_group_recommendations
def predict_health_risk(pm25, pm10, no2, co, o3, so2, humidity, temperature, wind):
    # Tạo DataFrame với dữ liệu đầu vào
    input_data = pd.DataFrame([[pm25, pm10, no2, co, o3, so2, humidity, temperature, wind]],
                             columns=features)
    
    # Tiền xử lý
    input_processed = preprocessed_pipeline.transform(input_data)
    
    # Dự đoán
    risk_level = model.predict(input_processed)[0]
    
    # Xác định xác suất cho mỗi cấp độ rủi ro
    risk_probs = model.predict_proba(input_processed)[0]
    
    return risk_level, risk_probs